In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from scripts.analysis import *

In [4]:
path = "/data/output/rocs/"

## Individual Analysis

In [20]:
path_input_individual = f"{path}results_individual"
df_individual = get_df_from_path(path_input_individual)
df_individual

fit time    roc time  roc value
class id    model         run id                                    
valve id_00 MFCCMix M     0        147.282579   41.764812   0.978321
            Baseline      1       1086.438118   41.841480   0.703481
            MFCCMix S+C   2        157.394463   39.984109   0.781371
            Baseline      3       1081.636285   41.878306   0.711037
            MFCCMix M+C   4        160.105759   39.783148   0.790834
...                                       ...         ...        ...
            MFCCMix M+S   248      152.604399   42.409173   0.714427
            MFCCMix M+S+C 249      154.802360   43.317260   0.947320
            MFCCMix S     250      154.570388   43.439360   0.836876
                          251      154.132209   38.858208   0.477509
            MFCCMix M+S+C 252      154.776499  289.728334   0.497083

[253 rows x 3 columns]

In [21]:
df_slice = (slice(None), slice(None), slice(None))
# df_slice = ("valve", slice(None), ["Baseline", "MFCCMix M"])

In [22]:
index = []
values = ['roc value']
aggfunc = ['mean']
df_individual_agg = get_df_aggregate(df_individual, index, values, df_slice, aggfunc)
df_individual_agg

index      mean                                  \
model                        Baseline MFCCMix C MFCCMix M MFCCMix M+C   
class id  run id                                                        
all   all all     roc value  0.719759  0.721432  0.749223    0.706831   

                                                                  
model            MFCCMix M+S MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class id  run id                                                  
all   all all       0.665668      0.634355  0.731302    0.656093

In [23]:
index = ['class']
values = ['roc value']
df_individual_agg = get_df_aggregate(df_individual, index, values, df_slice)
df_individual_agg


mean                                              \
                 roc value                                               
model             Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class id  run id                                                         
valve all all     0.719759  0.721432  0.749223    0.706831    0.665668   

                                                      
                                                      
model            MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class id  run id                                      
valve all all         0.634355  0.731302    0.656093

In [24]:
index = ['class', 'id']
values = ['roc value']
df_individual_agg = get_df_aggregate(df_individual, index, values, df_slice)
df_individual_agg

mean                                              \
                   roc value                                               
model               Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class id    run id                                                         
valve id_00 all     0.730203  0.721432  0.749223    0.706831    0.665668   
      id_06 all     0.695659       NaN       NaN         NaN         NaN   

                                                        
                                                        
model              MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class id    run id                                      
valve id_00 all         0.634355  0.731302    0.656093  
      id_06 all              NaN       NaN         NaN

In [25]:
index = ['class', 'id', 'run id']
values = ['roc value']
df_individual_agg = get_df_aggregate(df_individual, index, values, df_slice)
df_individual_agg

mean                                              \
                   roc value                                               
model               Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class id    run id                                                         
valve id_00 0            NaN       NaN  0.978321         NaN         NaN   
            1       0.703481       NaN       NaN         NaN         NaN   
            2            NaN       NaN       NaN         NaN         NaN   
            3       0.711037       NaN       NaN         NaN         NaN   
            4            NaN       NaN       NaN    0.790834         NaN   
...                      ...       ...       ...         ...         ...   
      id_06 156     0.503990       NaN       NaN         NaN         NaN   
            166     0.465363       NaN       NaN         NaN         NaN   
            197     0.919850       NaN       NaN         NaN         NaN   
            209     0.949439       NaN       NaN         NaN         NaN   
            247     0.336346       NaN       NaN         NaN         NaN   

                                                        
                                                        
model              MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class id    run id                                      
valve id_00 0                NaN       NaN         NaN  
            1                NaN       NaN         NaN  
            2                NaN       NaN    0.781371  
            3                NaN       NaN         NaN  
            4                NaN       NaN         NaN  
...                          ...       ...         ...  
      id_06 156              NaN       NaN         NaN  
            166              NaN       NaN         NaN  
            197              NaN       NaN         NaN  
            209              NaN       NaN         NaN  
            247              NaN       NaN         NaN  

[253 rows x 8 columns]

## Combined Analysis

In [8]:
path_input_combined = f"{path}results_combined"
df_combined = get_df_from_path(path_input_combined)
df_combined

fit time  roc time  roc value
class          id    model     run id                               
machine_type_1 id_00 MFCCMix M 0       1.071858  1.121786       1.00
               id_04 MFCCMix M 1       1.093579  1.105280       0.84
               id_00 MFCCMix M 2       2.890638  1.010204       0.68
               id_04 MFCCMix M 3       1.094739  1.116160       0.72
               id_02 MFCCMix M 4       1.094001  1.133934       0.64
               id_04 MFCCMix M 5       1.122451  1.324407       0.76
               id_02 MFCCMix M 6       1.091670  1.078403       0.88
               id_00 MFCCMix M 7       1.113179  1.147081       0.84
               id_02 MFCCMix M 8       1.154819  1.117413       0.88

In [50]:
index = []
values = ['roc value']
df_combined_agg = get_df_aggregate(df_combined, index, values, df_slice)
df_combined_agg

index      mean                                  \
model                        Baseline MFCCMix C MFCCMix M MFCCMix M+C   
class id  run id                                                        
all   all all     roc value  0.576767  0.535295  0.581274     0.59613   

                                                                  
model            MFCCMix M+S MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class id  run id                                                  
all   all all       0.585171      0.598701  0.569857    0.554155

In [56]:
index = ['class']
values = ['roc value']
df_combined_agg = get_df_aggregate(df_combined, index, values, df_slice)
df_combined_agg

mean                                              \
                  roc value                                               
model              Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class  id  run id                                                         
fan    all all     0.480623  0.547823  0.626480    0.626231    0.600869   
pump   all all     0.597953  0.623581  0.608585    0.633559    0.555636   
slider all all     0.728740  0.615290  0.599097    0.619778    0.640125   
valve  all all     0.499751  0.354487  0.490935    0.504953    0.544052   

                                                       
                                                       
model             MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class  id  run id                                      
fan    all all         0.661890  0.526168    0.536053  
pump   all all         0.596907  0.571933    0.583847  
slider all all         0.678314  0.686557    0.701620  
valve  all all         0.457692  0.494769    0.395101

In [58]:
index = ['class', 'id']
values = ['roc value']
df_combined_agg = get_df_aggregate(df_combined, index, values, df_slice)
df_combined_agg

mean                                              \
                   roc value                                               
model               Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class  id   run id                                                         
fan    id00 all     0.584497  0.573109  0.596518    0.573095    0.564000   
       id02 all     0.423729  0.627982  0.613688    0.605023    0.626012   
       id04 all     0.596963  0.543150  0.604893    0.597368    0.568448   
       id06 all     0.317301  0.447050  0.690820    0.729437    0.645018   
pump   id00 all     0.752441  0.726392  0.864019    0.843823    0.781391   
       id02 all     0.782485  0.612991  0.764115    0.767064    0.705814   
       id04 all     0.227800  0.471100  0.209800    0.239767    0.295733   
       id06 all     0.629085  0.683840  0.596405    0.683583    0.439607   
slider id00 all     0.978015  0.954375  0.617814    0.686940    0.794894   
       id02 all     0.733951  0.756966  0.741395    0.758074    0.772139   
       id04 all     0.637619  0.225098  0.598557    0.637388    0.549078   
       id06 all     0.565375  0.524723  0.438623    0.396709    0.444388   
valve  id00 all     0.475626  0.325354  0.495892    0.522962    0.577761   
       id02 all     0.563310  0.615486  0.533819    0.570602    0.598241   
       id04 all     0.498426  0.219699  0.559375    0.511181    0.536898   
       id06 all     0.461644  0.257407  0.374653    0.415069    0.463310   

                                                        
                                                        
model              MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class  id   run id                                      
fan    id00 all         0.577375  0.547127    0.564575  
       id02 all         0.720116  0.590276    0.587268  
       id04 all         0.649546  0.467948    0.506677  
       id06 all         0.700522  0.499323    0.485690  
pump   id00 all         0.831630  0.674018    0.714444  
       id02 all         0.776587  0.471039    0.493493  
       id04 all         0.144400  0.508433    0.486767  
       id06 all         0.635012  0.634243    0.640683  
slider id00 all         0.822105  0.984477    0.980077  
       id02 all         0.792521  0.753249    0.727952  
       id04 all         0.618220  0.504903    0.433594  
       id06 all         0.480411  0.503598    0.664857  
valve  id00 all         0.477156  0.631947    0.328390  
       id02 all         0.564815  0.624120    0.571690  
       id04 all         0.444398  0.377222    0.417639  
       id06 all         0.344398  0.345787    0.262685

In [12]:
index = ['class', 'id', 'run id']
df_combined_agg = get_df_aggregate(df_combined, index)
df_combined_agg

mean                                              \
model               Baseline MFCCMix C MFCCMix M MFCCMix M+C MFCCMix M+S   
class  id   run id                                                         
fan    id00 0       0.589753  0.572602  0.580963    0.574480    0.558621   
            1       0.584290  0.582110  0.612053    0.563475    0.548775   
            2       0.579448  0.564616  0.596538    0.581332    0.584604   
       id02 0       0.578068  0.631955  0.615645    0.550128    0.608996   
            1       0.329133  0.617314  0.625585    0.649576    0.683328   
            2       0.363987  0.634679  0.599832    0.615366    0.585711   
       id04 0       0.589130  0.558693  0.510313    0.598106    0.615900   
            1       0.584605  0.536118  0.657014    0.610830    0.539784   
            2       0.617156  0.534640  0.647353    0.583168    0.549660   
       id06 0       0.319595  0.425588  0.680121    0.716677    0.649243   
            1       0.305177  0.446697  0.757568    0.788538    0.572755   
            2       0.327131  0.468865  0.634771    0.683098    0.713055   
pump   id00 0       0.756663  0.729473  0.845127    0.846985    0.821654   
            1       0.753533  0.730500  0.886303    0.856521    0.759157   
            2       0.747127  0.719204  0.860629    0.827962    0.763363   
       id02 0       0.769337  0.542975  0.767146    0.744420    0.694668   
            1       0.786056  0.667965  0.704164    0.798231    0.712036   
            2       0.792062  0.628033  0.821037    0.758542    0.710738   
       id04 0       0.328800  0.486500  0.259900    0.193500    0.258700   
            1       0.222700  0.471600  0.209800    0.192000    0.285500   
            2       0.131900  0.455200  0.159700    0.333800    0.343000   
       id06 0       0.603422  0.680219  0.577470    0.673683    0.261246   
            1       0.593522  0.689158  0.557958    0.700404    0.459439   
            2       0.690311  0.682141  0.653787    0.676663    0.598135   
slider id00 0       0.979177  0.969961  0.524807    0.695177    0.712016   
            1       0.971168  0.961526  0.608667    0.649097    0.877896   
            2       0.983698  0.931637  0.719969    0.716545    0.794770   
       id02 0       0.736509  0.756133  0.646453    0.746006    0.837072   
            1       0.726311  0.740577  0.824405    0.762740    0.828809   
            2       0.739034  0.774187  0.753328    0.765476    0.650535   
       id04 0       0.646983  0.283077  0.584143    0.638714    0.610466   
            1       0.626720  0.190759  0.722952    0.660744    0.444925   
            2       0.639155  0.201458  0.488575    0.612707    0.591844   
       id06 0       0.538568  0.490595  0.451206    0.362707    0.397298   
            1       0.546522  0.523545  0.406262    0.395026    0.385431   
            2       0.611034  0.560030  0.458402    0.432395    0.550436   
valve  id00 0       0.327449  0.431961  0.507097    0.529765    0.438387   
            1       0.551868  0.310501  0.497140    0.529694    0.730316   
            2       0.547560  0.233599  0.483440    0.509427    0.564579   
       id02 0       0.589444  0.673889  0.540208    0.565278    0.590972   
            1       0.529306  0.582569  0.555347    0.576736    0.589028   
            2       0.571181  0.590000  0.505903    0.569792    0.614722   
       id04 0       0.511667  0.277222  0.531667    0.521667    0.578819   
            1       0.498472  0.136528  0.561806    0.554653    0.560764   
            2       0.485139  0.245347  0.584653    0.457222    0.471111   
       id06 0       0.394028  0.257083  0.425625    0.434167    0.506042   
            1       0.518611  0.293542  0.341736    0.411597    0.447778   
            2       0.472292  0.221597  0.356597    0.399444    0.436111   

                                                        
model              MFCCMix M+S+C MFCCMix S MFCCMix S+C  
class  id   run id               